# Basic steps for developing a model with Pycaret

In [1]:
#!pip install pycaret

In [2]:
from pycaret.datasets import get_data
data1 = get_data(dataset = 'traffic')

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,Rush Hour,traffic_volume
0,None,288.28,0.0,0.0,40,Clouds,1,5545
1,None,289.36,0.0,0.0,75,Clouds,0,4516
2,None,289.58,0.0,0.0,90,Clouds,0,4767
3,None,290.13,0.0,0.0,90,Clouds,0,5026
4,None,291.14,0.0,0.0,75,Clouds,0,4918


In [3]:
from pycaret.regression import *

In [5]:
dataset = setup(data = data1, target = 'traffic_volume',
                session_id = 438)

,Description,Value
0,Session id,438
1,Target,traffic_volume
2,Target type,Regression
3,Original data shape,"(48204, 8)"
4,Transformed data shape,"(48204, 29)"
5,Transformed train set shape,"(33742, 29)"
6,Transformed test set shape,"(14462, 29)"
7,Numeric features,5
8,Categorical features,2
9,Preprocess,True


In [6]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,1502.4277,3077063.5115,1754.0547,0.2189,0.9297,2.4635,0.3050
gbr,Gradient Boosting Regressor,1530.7769,3148984.7500,1774.4289,0.2006,0.9421,2.5603,1.0710
ada,AdaBoost Regressor,1577.2863,3282090.0319,1811.5740,0.1669,0.9730,2.7074,0.5000
knn,K Neighbors Regressor,1573.7193,3662877.1551,1913.7609,0.0702,0.9602,2.5605,0.3820
rf,Random Forest Regressor,1547.0023,3740449.9683,1933.7739,0.0504,0.9554,2.4869,4.6970
omp,Orthogonal Matching Pursuit,1712.8788,3848922.2942,1961.8288,0.0229,1.0224,2.9640,0.1200
dummy,Dummy Regressor,1743.7632,3939897.8403,1984.8734,-0.0001,1.0318,2.9022,0.1150
et,Extra Trees Regressor,1679.3206,4592693.0108,2142.8898,-0.1661,1.0486,2.6657,2.9070
dt,Decision Tree Regressor,1749.2798,5217471.6829,2283.9553,-0.3246,1.1263,2.5748,0.1530
huber,Huber Regressor,1611.7679,8073109.9280,2363.8425,-1.0665,0.9607,2.6034,0.7920


In [7]:
tuned_best = tune_model(estimator = best)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1486.2811,3054604.8890,1747.7428,0.2134,0.9327,5.0459
1,1509.9101,3123112.3214,1767.2330,0.2099,0.9357,2.6766
2,1503.5700,3098646.5240,1760.2973,0.2193,0.9388,2.1946
3,1490.2079,3054327.4922,1747.6634,0.2206,0.9207,3.0757
4,1512.5551,3115128.6707,1764.9727,0.2296,0.9286,1.7317
5,1495.9241,3055230.1784,1747.9217,0.2144,0.9304,2.7295
6,1458.1523,2942615.7706,1715.4054,0.2433,0.9032,1.5192
7,1508.3671,3130499.9015,1769.3219,0.1975,0.9388,2.3379
8,1528.9150,3204669.4930,1790.1591,0.1847,0.9319,1.5767


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [8]:
tuned_best = tune_model(estimator = best, fold = 5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1500.6461,3096046.8397,1759.5587,0.2099,0.9346,3.8403
1,1500.1358,3090278.3827,1757.9188,0.2164,0.9301,2.6539
2,1505.6980,3086485.7795,1756.8397,0.2219,0.9290,2.2074
3,1485.1463,3041877.7735,1744.0980,0.2190,0.9211,1.9294
4,1533.7358,3212596.4749,1792.3717,0.1915,0.9334,1.8772
Mean,1505.0724,3105457.0501,1762.1574,0.2118,0.9297,2.5016
Std,15.8934,56921.0574,16.0751,0.0109,0.0048,0.7238


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [9]:
final_model = finalize_model(estimator = tuned_best) 

In [14]:
save_model(final_model, '/2_Automating the ML Model Cycle/save_model/rf_base_traffic')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['temp', 'rain_1h', 'snow_1h',
                                              'clouds_all', 'Rush Hour'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['holiday', 'weather_main'],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('onehot_encoding',
                  TransformerWrapper(include=['holiday', 'weather_main'],
                                     transformer=OneHotEncoder(cols=['holiday',
                                                                     'weather_main'],
                                                               handle_missing='return_nan',
                                                               use_cat_names=True))),
                 ('clean_column_names',
 

#### Predictions

In [15]:
saved_model = load_model('/2_Automating the ML Model Cycle/save_model/rf_base_traffic')

Transformation Pipeline and Model Successfully Loaded


In [17]:
import pandas as pd

# Example new data
new_data = pd.DataFrame({
    'holiday': ['None', 'None'],
    'temp': [292.15, 293.10],  # example temperatures
    'rain_1h': [0.0, 0.25],  # example rain values
    'snow_1h': [0.0, 0.0],  # example snow values
    'clouds_all': [40, 50],  # example cloud coverage percentages
    'weather_main': ['Clear', 'Rain'],  # example weather conditions
    'Rush Hour': [1, 0]  # example rush hour flags
})


In [18]:
predictions = predict_model(saved_model, data=new_data)
predictions.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,Rush Hour,prediction_label
0,None,292.149994,0.00,0.0,40,Clear,1,4571.114853
1,None,293.100006,0.25,0.0,50,Rain,0,2592.517413


#### Comparision between Actual Label and Predicted Label (inlcuding dataset with Actual label)

In [19]:
import pandas as pd

# Example new data with actual labels
new_data_with_actual = pd.DataFrame({
    'holiday': ['None', 'None'],
    'temp': [292.15, 293.10],  # example temperatures
    'rain_1h': [0.0, 0.25],  # example rain values
    'snow_1h': [0.0, 0.0],  # example snow values
    'clouds_all': [40, 50],  # example cloud coverage percentages
    'weather_main': ['Clear', 'Rain'],  # example weather conditions
    'Rush Hour': [1, 0],  # example rush hour flags
    'traffic_volume': [5500, 4600]  # actual traffic volume
})


In [20]:
from pycaret.regression import load_model

saved_model = load_model('/2_Automating the ML Model Cycle/save_model/rf_base_traffic')

Transformation Pipeline and Model Successfully Loaded


In [26]:
new_data_for_prediction = new_data_with_actual.drop(columns=['traffic_volume'])
predictions = predict_model(saved_model, data=new_data_for_prediction)

In [28]:
print(predictions.columns)

Index(['holiday', 'temp', 'rain_1h', 'snow_1h', 'clouds_all', 'weather_main',
       'Rush Hour', 'prediction_label'],
      dtype='object')


In [29]:
# Add the predictions to the original DataFrame with actual labels
new_data_with_actual['predicted_traffic_volume'] = predictions['prediction_label']

# Now you can compare
print(new_data_with_actual[['traffic_volume', 'predicted_traffic_volume']])


   traffic_volume  predicted_traffic_volume
0            5500               4571.114853
1            4600               2592.517413


In [30]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Actual and predicted values
actual_values = new_data_with_actual['traffic_volume']
predicted_values = new_data_with_actual['predicted_traffic_volume']

# Calculate MAE, MSE, RMSE, and R-squared
mae = mean_absolute_error(actual_values, predicted_values)
mse = mean_squared_error(actual_values, predicted_values)
rmse = np.sqrt(mse)
r2 = r2_score(actual_values, predicted_values)

# Print the performance metrics
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")


Mean Absolute Error (MAE): 1468.183867272633
Mean Squared Error (MSE): 2446406.977294451
Root Mean Squared Error (RMSE): 1564.0994141340414
R-squared (R²): -11.081022110096054
